In [ ]:
#install TelegramBot
!pip install pyTelegramBotAPI
!pip install transformers
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.0/265.0 kB 7.0 MB/s eta 0:00:00


# New section

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel,AutoTokenizer, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

def process_csv(file_path):
    df = pd.read_csv(file_path)
    qa_pairs = []

    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")

    return qa_pairs

def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True,
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token


# Load and preprocess the dataset
train_dataset = load_dataset("/content/questions.csv", tokenizer)
valid_dataset = load_dataset("/content/valid.csv", tokenizer)

# Configure and train the model using the Trainer class
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

trainer.train()


FileNotFoundError: [Errno 2] No such file or directory: '/content/questions.csv'

In [ ]:
import telebot
bot = telebot.TeleBot("7917921260:AAG_sezdUwHW-zwy-J0xYA3iWxU__Qg232c")

In [ ]:
# Save the fine-tuned model
model.save_pretrained("Heart_Disease_Prediction_gpt2")


In [ ]:
import telebot
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("Heart_Disease_Prediction_gpt2")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

API_TOKEN = '7917921260:AAG_sezdUwHW-zwy-J0xYA3iWxU__Qg232c'
bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start'])
def handle_start(message):
  bot.reply_to(message, "Hello Ask any questions about, I will give the best answer.")

@bot.message_handler(func=lambda message: True)
def answer_question(message):
  question=message.text

  answer=ask_question(question,fine_tuned_model,tokenizer)

  bot.send_message(message.chat.id,answer)

def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()

    # Truncate the answer after the first newline character
    answer = answer.split("\n")[0]

    return answer


# Ask questions using the fine-tuned model
question = "What is Heart disease"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

question = "What type of disease is this"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

bot.polling()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Question: What is Heart disease
Answer: Heart disease is the most common and disabling condition in the body. Heart diseases occur due to the immune system failing to respond to the body's demands. Because the body is able to respond without a response, it is called "normal".
Question: What type of disease is this
Answer: This is the type of sickness that is known as C. elegans, or the cadaver-fish. This is a parasitic disease that occurs when a parasite attaches to a tissue of the body. This infection can happen anywhere in the body including on the skin and in the stomach. It affects many people, even if not from this illness. There are many theories on this. One is that C. fowleri is due to the parasite attaching to the liver and causing it to form a problem with the liver itself. This problem is not specific to humans and is not present


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:

question = "How to succeed in life"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

NameError: name 'ask_question' is not defined

In [ ]:
question = "how to deal with depression and stress"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

NameError: name 'ask_question' is not defined

In [ ]:
question = "How to deal with anger"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

NameError: name 'ask_question' is not defined